# Set-up

In [2]:
import json
import pandas as pd
from jinja2 import Environment, FileSystemLoader
from jinja2.exceptions import UndefinedError
from pathlib import Path
import shutil
from datetime import datetime, date

In [4]:
TEMPLATE_DIR_PATH = Path("../templates")

Function to create proper HTML file names by replacing spaces with underscores

In [ ]:
def page_link(a):
    """Return the HTML file name after replacing blank spaces(" ") with underscores("-")"""
    return a.replace(" ", "_") if " " in a else a

Creating an instance of the Environment class that looks for templates. Page_link is set to the global variable so that it can be accessed by all templates

In [ ]:
environment = Environment(
    loader=FileSystemLoader(TEMPLATE_DIR_PATH),
    extensions=["jinja2.ext.loopcontrols", "jinja2.ext.do"],
)
environment.globals["page_link"] = page_link